# Proyecto 2 - regresion logistica


# 1. Entendimiento de los datos

In [1]:
#Instalaciones

#Libreria para remplazar numeros por palabras
!pip install num2words
#Instalar  pandas profiler
!pip install pandas-profiling==2.7.1
#deteccion de lenguaje para eliminar entradas que no esten en español
!pip install langdetect
#Procesamientno de lenguaje natural en español
!pip install stanza

ERROR: Could not find a version that satisfies the requirement pandas-profiling==2.7.1 (from versions: 3.0.0, 3.1.0, 3.2.0, 3.3.0, 3.4.0, 3.5.0, 3.6.0, 3.6.1, 3.6.2, 3.6.3, 3.6.4, 3.6.5, 3.6.6)
ERROR: No matching distribution found for pandas-profiling==2.7.1


In [2]:
#Imports generales para analisis de datos y ML
import pandas as pd
import numpy as np
import sys
from pandas_profiling import ProfileReport
import statistics

import sklearn
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

C:\Users\andre\AppData\Local\Temp\ipykernel_17764\122007551.py:5: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


# Leer los datos

In [3]:
# Lectura de los datos.
df = pd.read_excel("Completos.xlsx")

df.head()

,Id,Basura,EnfermedadesRespiratorias,Localidad,Año,Estado,Quema
0,1,Servicio aseo,Si,SUBA,2017,Bueno,No quema
1,2,Servicio aseo,Si,USAQUEN,2017,Regular,No quema
2,3,Servicio aseo,Si,USAQUEN,2017,Bueno,No quema
3,4,Servicio aseo,Si,USAQUEN,2017,Bueno,No quema
4,5,Servicio aseo,Si,SUBA,2017,Regular,No quema


# Preparación

In [4]:
#Mapear variables
df['EnfermedadesRespiratorias'] = df['EnfermedadesRespiratorias'].map({'Si': 1, 'No': 0})
df['Estado'] = df['Estado'].map({'Muy bueno': 1, 'Bueno': 2, 'Regular': 3, 'Malo': 4, 'Muy malo': 5})
df['Quema'] = df['Quema'].map({'No quema': 0, 'Quema': 1})

df.head()


,Id,Basura,EnfermedadesRespiratorias,Localidad,Año,Estado,Quema
0,1,Servicio aseo,1,SUBA,2017,2,0
1,2,Servicio aseo,1,USAQUEN,2017,3,0
2,3,Servicio aseo,1,USAQUEN,2017,2,0
3,4,Servicio aseo,1,USAQUEN,2017,2,0
4,5,Servicio aseo,1,SUBA,2017,3,0


In [5]:
df['Interaction'] = df['EnfermedadesRespiratorias'] * df['Quema']

In [6]:
X = df[['EnfermedadesRespiratorias', 'Quema', 'Interaction']]
y = df['Estado']

# Modelo

## Regresion logistica

In [7]:
from matplotlib import style
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import statsmodels.api as sm


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
#Implementar la regresin con k-fold cross validation
model_mnlogit = sm.MNLogit(y_train, sm.add_constant(X_train))
result = model_mnlogit.fit()


Optimization terminated successfully.
         Current function value: 0.840376
         Iterations 8


In [10]:
print(result.summary())

                          MNLogit Regression Results                          
Dep. Variable:                 Estado   No. Observations:               489786
Model:                        MNLogit   Df Residuals:                   489770
Method:                           MLE   Df Model:                           12
Date:                Fri, 01 Dec 2023   Pseudo R-squ.:                 0.01432
Time:                        12:56:29   Log-Likelihood:            -4.1160e+05
converged:                       True   LL-Null:                   -4.1758e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                 Estado=2       coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                         1.4789      0.004    368.321      0.000       1.471       1.487
EnfermedadesRespiratorias     0.4203      0.030     13.820      0.000       0.361     

In [17]:
from tabulate import tabulate

In [21]:
# Extract coefficients and p-values directly from the result object
summary_df = result.summary2().tables[1]

# Print the table using tabulate
print(tabulate(summary_df, headers='keys', tablefmt='pretty'))

+---------------------------+---------------------------+----------------------+----------------------+---------------------+------------------------+---------------------+---------------------+
|                           |        Estado = 0         |        Coef.         |       Std.Err.       |          t          |         P>|t|          |       [0.025        |       0.975]        |
+---------------------------+---------------------------+----------------------+----------------------+---------------------+------------------------+---------------------+---------------------+
|           const           |           const           |  1.4789122022985766  | 0.004015276512074041 | 368.32138405697566  |          0.0           |  1.471042404946942  | 1.4867819996502112  |
| EnfermedadesRespiratorias | EnfermedadesRespiratorias |  0.4203004530842911  | 0.030413244997406667 | 13.819651705042657  | 1.9400624545081453e-43 | 0.36069158823638103 | 0.47990931793220115 |
|           Quema        